<a href="https://colab.research.google.com/github/terryspitz/ipython_notebooks/blob/master/deeper/deeperfont.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install dependencies for colab.research.google.com.
!pip install -I tensorflow==1.11.0
!pip install tensorflow-probability-gpu
!pip install dm-sonnet
#!pip install tfp-nightly

# Install dependencies for GCP Deep Learning VM.
! pip install dm-sonnet
! pip install tensorflow-probability-gpu
! pip install wrapt

    100% |████████████████████████████████| 63.0MB 521kB/s 
    100% |████████████████████████████████| 3.0MB 8.6MB/s 
    100% |████████████████████████████████| 573kB 16.4MB/s 
    100% |████████████████████████████████| 9.7MB 1.1MB/s 
    100% |████████████████████████████████| 51kB 20.2MB/s 
    100% |████████████████████████████████| 1.1MB 12.4MB/s 
    100% |████████████████████████████████| 102kB 27.9MB/s 
    100% |████████████████████████████████| 13.9MB 2.4MB/s 
    100% |████████████████████████████████| 92kB 26.9MB/s 
    100% |████████████████████████████████| 327kB 21.2MB/s 
    100% |████████████████████████████████| 2.8MB 8.7MB/s 
  Running setup.py bdist_wheel for gast ... - done
  Stored in directory: /root/.cache/pip/wheels/9a/1f/0e/3cde98113222b853e98fc0a8e9924480a3e25f1b4008cedb4f
  Running setup.py bdist_wheel for termcolor ... - done
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py b

In [3]:
from enum import Enum, IntEnum
import math
import os
import time
import numpy as np
print(np.__version__)
import tensorflow as tf
print(tf.__version__)
import sonnet as snt
print(snt.__version__)
np.set_printoptions(precision=2)
from PIL import Image, ImageDraw, ImageChops, ImageFont
from IPython.display import display

# hosted colab versions
# np 1.14.6
# tf 1.11.0
# snt 1.23

1.14.6


ImportError: ignored

In [0]:
class Stroke(IntEnum):
    """
    Enum defining data contents of last dimension
    """
    DX = 0
    DY = 1
    PenDown = 2  # +1 draw stroke, 0 if this stroke is hidden, i.e. to move between contours in the glyph
    EndOfGlyph = 3  # +1 indicates this stroke is the last of the current glyph, else 0
    Size = 4  # size of this enum

In [0]:
def drawOutlines(outlines, cellsize, glyphCount):
  """Draw outlines from array."""
  im = Image.new('1', size=(glyphCount*cellsize, outlines.shape[0]*cellsize), color=(0))
  def drawPoints(im, points):
    if len(points)<=2:
      return im
    #temporary image to use to xor each part with main image
    im2 = Image.new('1', size=(glyphCount*cellsize, outlines.shape[0]*cellsize), color=(0)) 
    draw = ImageDraw.Draw(im2).polygon(points, fill=1)
    return ImageChops.logical_xor(im, im2)
  scale = cellsize * 0.9
  for font in range(outlines.shape[0]):
    i = 0
    for glyphNum in range(glyphCount):
      points = []
      x,y = (glyphNum * cellsize, (font+1) * cellsize)
      #ImageDraw.Draw(im).line((glyphNum * cellsize, font * cellsize, glyphNum * cellsize, (font+1) * cellsize), fill=1)
      #ImageDraw.Draw(im).line((glyphNum * cellsize, font * cellsize, (glyphNum+1) * cellsize, font * cellsize), fill=1)
      while True:
        x += outlines[font,i,Stroke.DX]*scale
        y -= outlines[font,i,Stroke.DY]*scale
        #print(outlines[font,i], x,y)
        if outlines[font,i,Stroke.PenDown]>0.5:
          points += (x,y)
        else:
          im = drawPoints(im, points)
          points=[(x,y)]
        i+=1
        if outlines[font,i-1,Stroke.EndOfGlyph]>0.5 or i>=outlines.shape[1]:
          break
      im = drawPoints(im, points)
      if i>=outlines.shape[1]:
        break
  return im

In [0]:
def outlinesToScanlines(outlines, max_points_per_scanline, ygrid, glyphCount):
    '''
    Pure tensorflow function
    Input: 2D tensor of max_outline_points * Stroke (DX,DY,PenDown,EndOfGlyph)
    Output: glyphCount * y coord * sorted x coords for each scanline
    '''
    glyphNums = tf.cast(outlines[:-1,Stroke.EndOfGlyph]>0.5, tf.int32)
    glyphNums = tf.cumsum(glyphNums, axis=-1)
    glyphNums = tf.concat((tf.zeros([1], dtype=tf.int32), glyphNums), axis=0)
    glyphNums2D = tf.expand_dims(tf.range(glyphCount),-1)
    glyphNums2D = tf.tile(glyphNums2D, [1,glyphNums.shape[0]])
    glyphNums2D = tf.math.equal(glyphNums2D, glyphNums)
    glyphNums2D = tf.cast(glyphNums2D, tf.float32)
    xs = tf.cumsum(outlines[:,Stroke.DX] * glyphNums2D, axis=1)+1e-2
    ys = tf.cumsum(outlines[:,Stroke.DY] * glyphNums2D, axis=1)+1e-2
    #offset to create line start x1,y1 to end x2,y2
    #add 3rd dimension (size=1) so following interpolation in y is broadcast across all lines
    x1s = tf.expand_dims(xs[:,:-1], -1)
    x2s = tf.expand_dims(xs[:,1:], -1)
    y1s = tf.expand_dims(ys[:,:-1], -1)
    y2s = tf.expand_dims(ys[:,1:], -1)
    #linear interpolate the x coords for all lines at all y coord
    xxs = x1s + (x2s-x1s)*(ygrid-y1s)/(y2s-y1s)
    #zero x coords outside of line y bounds or where dy=0 and for hidden lines
    in_range = tf.logical_or(tf.logical_and(y1s<ygrid, ygrid<=y2s), tf.logical_and(y2s<ygrid, ygrid<=y1s))
    in_range = tf.logical_and(in_range, y2s!=y1s)
    visible = outlines[1:,Stroke.PenDown]>0.5
    in_range = tf.logical_and(in_range, tf.expand_dims(visible,-1))
    xxs = tf.where(in_range, xxs, tf.zeros_like(xxs)) 
    return tf.nn.top_k(tf.transpose(xxs, perm=(0,2,1)), max_points_per_scanline).values
  
def drawScanLines(xxs, ygrid, cellsize):
    '''Draw a glyph rasterisation based on a input y coord array with array of x-intercepts of lines with the y coord'''
    columns = math.ceil(1000/cellsize)
    rows = 1 #math.ceil(xxs.shape[0]/columns)
    scale = cellsize * 0.9
    im = Image.new('1', size=(columns*cellsize, rows*cellsize), color=(0)) 
    draw = ImageDraw.Draw(im) 
    for i in range(xxs.shape[0]):
      yy,xx = divmod(i, columns)
      xx *= cellsize
      yy = (yy+1)*cellsize-1
      for xs, y in zip(xxs[i], ygrid):
        for j in range(0, xs.shape[0], 2):
          if(xs[j]>1e-5):
            if(xs[j+1]>1e-5):
              draw.line((xx+xs[j]*scale, yy-y*scale, xx+xs[j+1]*scale, yy-y*scale), fill=1)
            else:
              draw.ellipse((xx+xs[j]*scale-1, yy-y*scale-1, xx+xs[j]*scale+1, yy-y*scale+1), fill=1)
    return im  

In [7]:
def parse_proto(example_proto):
  features = {
    'glyphNum': tf.FixedLenFeature([1], tf.int64),
    'max_outline_points': tf.FixedLenFeature([1], tf.int64),
    'outline': tf.FixedLenFeature([5000*4], tf.float32),
  }
  return tf.parse_single_example(example_proto, features)

#filename = '/content/gdrive/My Drive/Colab Notebooks/fonts8.tfrecords'
filename = '/home/jupyter/ipython_notebooks/deeper/fonts/fonts8.tfrecords'
dataset = tf.data.TFRecordDataset(filename).map(parse_proto)

NameError: ignored

In [8]:
# Plot first row of original data
dataset_iter = dataset.make_one_shot_iterator()
batch = dataset_iter.get_next()

with tf.Session():
  glyphNum = int(batch["glyphNum"].eval()[0])
  max_outline_points = batch["max_outline_points"].eval()[0]
  outlines = tf.reshape(batch["outline"], (-1, max_outline_points, Stroke.Size)).eval()
  print(outlines.shape)
  assert glyphNum == 63
  assert max_outline_points == 5000
  assert outlines.shape[0] == 1

outlines = outlines[:,0:200,:]
print("outline shape (fonts, points_per_outline, Stroke.Size): ", outlines.shape)
display(drawOutlines(outlines, cellsize=30, glyphCount=glyphNum))

with tf.Session():
  y_divisions = 30
  max_points_per_scanline = 8
  ygrid = np.linspace(0.0, 1.0, y_divisions, endpoint=False) #y coordinates to render on
  scanlines = outlinesToScanlines(outlines[0], max_points_per_scanline, ygrid, glyphNum)
  print("scanlines shape (glyphs, lines, max_points_per_line): ", scanlines.shape)
  display(drawScanLines(scanlines.eval(), ygrid, 30))
  

(1, 5000, 4)
outline shape (fonts, points_per_outline, Stroke.Size):  (1, 200, 4)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


scanlines shape (glyphs, lines, max_points_per_line):  (63, 30, 8)


In [14]:
def encoder(x, latent_dim, hidden_size):
  """Construct an inference network parametrizing a Gaussian.
  Args:
    x: Outlines.
    latent_dim: The latent dimensionality.
    hidden_size: The size of the neural net hidden layers.
  Returns:
    mu: Mean parameters for the variational family Normal
    sigma: Standard deviation parameters for the variational family Normal
  """
  with tf.name_scope("encoder"):
    flat = snt.BatchFlatten()(x)
    hidden1 = tf.nn.elu(snt.Linear(hidden_size)(flat))
    hidden2 = tf.nn.elu(snt.Linear(hidden_size)(hidden1))
    latent_mean = snt.Linear(latent_dim)(hidden2)  # The mean parameter is unconstrained
    latent_log_var = snt.Linear(latent_dim)(hidden2)
    return latent_mean, latent_log_var


def decoder(z, hidden_size):
  """Build a generative network parametrizing the likelihood of the data
  Args:
    z: Samples of latent variables
    hidden_size: Size of the hidden state of the neural net
  Returns:
    gaussian_logits for the 2x real variables
    bernoulli_logits for the likelihood of the 2x bool data
  """
  with tf.name_scope("decoder"):
    hidden1 = tf.nn.elu(snt.Linear(hidden_size)(z))
    hidden2 = tf.nn.elu(snt.Linear(hidden_size)(hidden1))
    output = snt.Linear(np.prod(posterior_sample_shape))(hidden2)
    logits = snt.BatchReshape(posterior_sample_shape)(output) # Q: are these really logits?
    p_x_given_z1 = tf.distributions.Normal(loc=logits[...,0:2], 
                                           scale=tf.sqrt(tf.clip_by_value(tf.exp(logits[...,2:4]), 1e-2, 2.0)),
                                           validate_args=True, allow_nan_stats=False)
    p_x_given_z2 = tf.distributions.Bernoulli(logits=logits[...,4:6],
                                              validate_args=True, allow_nan_stats=False)
    return tf.concat([p_x_given_z1.sample(), tf.to_float(p_x_given_z2.sample())], axis=-1), logits, p_x_given_z1, p_x_given_z2


# https://github.com/altosaar/variational-autoencoder/blob/master/vae.py

def train_graph(x):
  with tf.name_scope('variational'):
    latent_mean, latent_log_var = encoder(x=x,
                                          latent_dim=FLAGS.latent_dim,
                                          hidden_size=FLAGS.hidden_size)
    latent_stdev = tf.sqrt(tf.exp(latent_log_var))
    q_z = tf.distributions.Normal(loc=latent_mean, scale=latent_stdev)
    assert q_z.reparameterization_type == tf.distributions.FULLY_REPARAMETERIZED

  with tf.name_scope('model'):
    # The likelihood is Bernoulli-distributed with logits given by the
    # generative decoder network
    posterior_predictive_samples, decoder_logits, p_x_given_z1, p_x_given_z2 = decoder(z=q_z.sample(), hidden_size=FLAGS.hidden_size)
    #tf.summary.image('posterior_predictive', posterior_predictive_samples)

    # Take samples from the prior
    p_z = tf.distributions.Normal(loc=np.zeros(FLAGS.latent_dim, dtype=np.float32),
                                  scale=np.ones(FLAGS.latent_dim, dtype=np.float32))
    prior_predictive_samples, _, _, _ = decoder(z=p_z.sample(FLAGS.n_samples), hidden_size=FLAGS.hidden_size)
    #tf.summary.image('prior_predictive', tf.cast(prior_predictive_samples, tf.float32))

    # Take samples from the prior with a placeholder
    #z_input = tf.placeholder(tf.float32, [None, FLAGS.latent_dim])
    #p_x_given_z_logits = decoder(z=z_input, hidden_size=FLAGS.hidden_size)
    #prior_predictive_inp = tf.distributions.Bernoulli(logits=p_x_given_z_logits)
    #prior_predictive_inp_sample = prior_predictive_inp.sample()

    # Build the evidence lower bound (ELBO) or the negative loss
    kl = tf.reduce_sum(tf.distributions.kl_divergence(q_z, p_z), 1)
    expected_log_likelihood1 = tf.reduce_sum(p_x_given_z1.log_prob(x[...,:Stroke.PenDown]), [1, 2])
    expected_log_likelihood2 = tf.reduce_sum(p_x_given_z2.log_prob(x[...,Stroke.PenDown:]), [1, 2])
    #global debug1, debug2, debug3
    #debug1 = tf.reduce_max(p_x_given_z1.log_prob(x[...,:Stroke.PenDown]))
    #debug2 = tf.reduce_max(p_x_given_z2.log_prob(x[...,Stroke.PenDown:]))
    #debug3 = kl
    elbo = tf.reduce_sum(expected_log_likelihood1 + expected_log_likelihood2 - kl, 0)
    
    # try optimising raw encoder/decoder
    # compare posterior 0:2 (mean) with DX/DY, and 4:6 (bernoulli p) with bool PenDown/EndOfGlyph
    raw_loss = tf.reduce_sum(kl) + tf.losses.mean_squared_error(x[...,0:2], decoder_logits[...,0:2]) + \
      tf.losses.mean_squared_error(x[...,2:4], decoder_logits[...,4:6])
    
    # Q: do we have any regularisation of encoder/decoder weights?
    optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)
    #return optimizer.minimize(-elbo), -elbo, posterior_predictive_samples, prior_predictive_samples
    return optimizer.minimize(raw_loss), raw_loss, posterior_predictive_samples, prior_predictive_samples

class FLAGS(): 
  logdir = "/tmp/tflog"
  iterations = 100000
  hidden_size = 100
  latent_dim = 64
  n_samples = 5
  print_every = 100
  batch_size = 100

batch_input_shape = [FLAGS.batch_size, max_outline_points, Stroke.Size]
outline_points = 200
input_shape = [outline_points, Stroke.Size]
posterior_sample_shape = [outline_points, 6] # 2x(mean, stdev, bool)
buffer_size=100
batched_dataset = dataset.shuffle(buffer_size)
batched_dataset = batched_dataset.repeat()
batched_dataset = batched_dataset.batch(FLAGS.batch_size)
batch = batched_dataset.make_one_shot_iterator().get_next()
outlines = tf.reshape(batch['outline'], batch_input_shape)
print(outlines.shape)
outlines = outlines[:,0:outline_points,:]
print(outlines.shape)
train_op, loss, posterior_predictive_samples, prior_predictive_samples = train_graph(outlines)

# Merge all the summaries
summary_op = tf.summary.merge_all()


/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


(100, 5000, 4)
(100, 200, 4)


In [0]:
# Run training
with tf.Session() as sess:
  init = tf.global_variables_initializer()
  sess.run(init)

  #print('Saving TensorBoard summaries and images to: %s' % FLAGS.logdir)
  #train_writer = tf.summary.FileWriter(FLAGS.logdir, sess.graph)

  t0 = time.time()
  for i in range(FLAGS.iterations+1):
    #print(loss.eval())

    #sess.run([tf.add_check_numerics_ops(), train_op])
    sess.run(train_op)

    # Print progress and save samples every so often
    if i % FLAGS.print_every == 0:
      np_loss = sess.run(loss)
      #train_writer.add_summary(summary_str, i)
      print('Iteration: {0:d} loss: {1:.3f} s/iter: {2:.3e}'.format(
          i,
          np_loss / FLAGS.batch_size  * 1e9,
          (time.time() - t0) / FLAGS.print_every))
      t0 = time.time()

      # Save samples
#       np_posterior_samples, np_prior_samples = sess.run(
#           [posterior_predictive_samples, prior_predictive_samples])
      # for k in range(FLAGS.n_samples):
    if i % (FLAGS.print_every*50) == 0:
      display(drawOutlines(posterior_predictive_samples.eval(), cellsize=10, glyphCount=glyphNum))
      #display(drawOutlines(np_prior_samples, cellsize=30, glyphCount=glyphNum))
      #f_name = os.path.join(FLAGS.logdir, 'iter_%d_posterior_predictive_%d_data.jpg' % (i, k))
      #imsave(f_name, np_x[k, :, :, 0])
      #f_name = os.path.join(FLAGS.logdir, 'iter_%d_posterior_predictive_%d_sample.jpg' % (i, k))
      #imsave(f_name, np_posterior_samples[k, :, :, 0])
      #f_name = os.path.join(FLAGS.logdir, 'iter_%d_prior_predictive_%d.jpg' % (i, k))
      #imsave(f_name, np_prior_samples[k, :, :, 0])


In [0]:
      # Plot the posterior predictive space
      #     if FLAGS.latent_dim == 2:
      #       np_q_mu = sess.run(q_mu, {x: np_x_fixed})
      #       cmap = mpl.colors.ListedColormap(sns.color_palette("husl"))
      #       f, ax = plt.subplots(1, figsize=(6 * 1.1618, 6))
      #       im = ax.scatter(np_q_mu[:, 0], np_q_mu[:, 1], c=np.argmax(np_y, 1), cmap=cmap,
      #                       alpha=0.7)
      #       ax.set_xlabel('First dimension of sampled latent variable $z_1$')
      #       ax.set_ylabel('Second dimension of sampled latent variable mean $z_2$')
      #       ax.set_xlim([-10., 10.])
      #       ax.set_ylim([-10., 10.])
      #       f.colorbar(im, ax=ax, label='Digit class')
      #       plt.tight_layout()
      #       plt.savefig(os.path.join(FLAGS.logdir,
      #                                'posterior_predictive_map_frame_%d.png' % i))
      #       plt.close()

      #       nx = ny = 20
      #       x_values = np.linspace(-3, 3, nx)
      #       y_values = np.linspace(-3, 3, ny)
      #       canvas = np.empty((28 * ny, 28 * nx))
      #       for ii, yi in enumerate(x_values):
      #         for j, xi in enumerate(y_values):
      #           np_z = np.array([[xi, yi]])
      #           x_mean = sess.run(prior_predictive_inp_sample, {z_input: np_z})
      #           canvas[(nx - ii - 1) * 28:(nx - ii) * 28, j *
      #                  28:(j + 1) * 28] = x_mean[0].reshape(28, 28)
      #       imsave(os.path.join(FLAGS.logdir,
      #                           'prior_predictive_map_frame_%d.png' % i), canvas)
      # plt.figure(figsize=(8, 10))
      # Xi, Yi = np.meshgrid(x_values, y_values)
      # plt.imshow(canvas, origin="upper")
      # plt.tight_layout()
      # plt.savefig()
